In [1]:
import sys
sys.path.append('../../')
import os
import pandas as pd
import numpy as np

# from pprint import pprint

from eztrack.edp.objects.clinical.clinical_object import ClinicalMeta
from eztrack.edp.loaders.dataset.clinical.excel_meta import ExcelReader
from eztrack.edp.objects.clinical.master_clinical import MasterClinicalSheet
from eztrack.edp.utils.utils import loadjsonfile


# Import magic commands for jupyter notebook 
# - autoreloading a module
# - profiling functions for memory usage and scripts
%load_ext autoreload
%autoreload 2

/Users/adam2392/anaconda2/envs/eztrack/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 88 from C header, got 96 from PyObject
  return f(*args, **kwds)


In [2]:
datadir = "/Users/adam2392/Dropbox/phd_research/Fragility_Analysis_Project/"
# datadir = "/home/adam2392/Documents/Dropbox/phd_research/Fragility_Analysis_Project/"

excelfilename = "organized_clinical_datasheet_raw.xlsx"
excelfilepath = os.path.join(datadir, excelfilename)

outputexcelfilename = "organized_clinical_datasheet_formatted.xlsx"
outputexcelfilepath = os.path.join(datadir, outputexcelfilename)
print(os.path.exists(excelfilepath))
print(excelfilepath)

True
/Users/adam2392/Dropbox/phd_research/Fragility_Analysis_Project/organized_clinical_datasheet_raw.xlsx


# 1. Perform Reading of Multiple Datasheets -> Save Formatted Excel Sheet Out

In [3]:
clinreader = ExcelReader(excelfilepath, expanding_semio=True)

In [4]:
datasetdf = clinreader.datasetdf

display(clinreader.ieegdf.head(2))
display(datasetdf.head(2))
display(clinreader.scalpdf.head(2))

,patient_id,number_datasets,clinical_center,modality,clinical_difficulty,engel_score,outcome,clinical_match,age_surgery,onset_age,...,resected_contacts,scalpeeg_localization,implantation_lobes,clinicalez_lobe,imaging_outcome_notes,bad_contacts,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,pt1,4.0,nih,ecog,1.0,1.0,s,1.0,30.0,13.0,...,"[att1, att2, att3, att4, att5, att6, att7, att...",likely dual pathology - r parietal and r mesia...,r temporal and parietal,"right temporal, right parietal",large area of encephalomalacia in r parietal r...,[],,,,
1,pt2,3.0,nih,ecog,1.0,1.0,s,1.0,28.0,19.0,...,"[g1, g2, g3, g4, g9, g10, g11, g12, g18, g19, ...",l temporal,l temporal,,l mesial temporal sclerosis,[],,,,


,patient_id,dataset_id,clinical_seizure_identifier,clinical_semiology,ez_hypo_contacts,onset_contacts,seizure_semiology,ez_hypo_notes,onset_brainregion
0,pt1,sz2,sz2,"complex partial, at times with secondary gener...","[pd1, pd2, pd3, pd4, ad1, ad2, ad3, ad4, att1,...",pd1-4; ad1-4; att1-2,"[[pd1, pd2, pd3, pd4, ad1, ad2, ad3, ad4, att1...",,r atl
1,pt1,sz3,sz3,,"[pd1, pd2, pd3, pd4, ad1, ad2, ad3, ad4, att1,...",pd1-4; ad1-4; att1-2,"[[pd1, pd2, pd3, pd4, ad1, ad2, ad3, ad4, att1...",,l atl


,patient_id,number_datasets,ieeg_patient_id,surgery_location,cezlobe,implantationlobe,imaging_procedures,imaging_notes,scalpeeg_clinical_localization,scalpeeg_clinical_localization(Shortened),Clinical_center,Position of elctrodes
0,pt3,3,pt3,r frontal topectomy,[right-frontal],r frontotemporal,mri,normal,r frontal,r frontal,nih,r frontotemporal
1,pt1,3,pt1,r atl,"[right-temporal, right-parietal]",r temporal and parietal,mri,large area of encephalomalacia in r parietal r...,likely dual pathology - r parietal and r mesia...,"r parietal, r mesial temporal",nih,r temporal and parietal


In [5]:
clinreader.write_to_excel(outputexcelfilepath)

# 2. Read in the Formatted Excel Sheet and Perform Various Queries

In [9]:
clinreader = ExcelReader(excelfilepath)

In [16]:
ieegdf, datasetdf, scalpdf = clinreader.read_formatted_df()

In [17]:
mastersheet = MasterClinicalSheet(ieegdf, datasetdf, scalpdf)

In [9]:
# patients = mastersheet.patients
# datasets = mastersheet.datasets

In [18]:
testpats = [
    'ummc007', 'la04', 'pt1', 'pt2', 'pt3', 'pt6', 'pt17'
]

for pat in testpats:
    print(mastersheet.get_patient_cezlobe(pat))
    print(mastersheet.get_patient_clinicaldiff(pat))
    print(mastersheet.get_patient_outcome(pat))
    print(mastersheet.get_patient_dataset_ezhypo(pat, 'sz3'))
    
    break

None
4
('nr', -1)


KeyError: 0

In [19]:
pat = 'pt1'
ezhypo = mastersheet.get_patient_dataset_ezhypo(pat, 'sz2')
print(ezhypo)

['pd1', 'pd2', 'pd3', 'pd4', 'ad1', 'ad2', 'ad3', 'ad4', 'att1', 'att2']


ValueError: malformed node or string: ['pd1', 'pd2', 'pd3', 'pd4', 'ad1', 'ad2', 'ad3', 'ad4', 'att1', 'att2']

In [ ]:
display(mastersheet.dataset_df[mastersheet.dataset_df.patient_id == 'pt1']['ez_hypo_contacts'].values[0])